In [1]:
import yahooquery
import requests
import json
from yahooquery import Ticker
import time
import json
import requests
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime

In [2]:
class StockDetail():

    api_key = 'ADLK0ZK57SB9BJKX'

    def __init__(self, symbol):

        self.symbol = symbol
    
    #returns earnings trend data
    def get_earnings_trend(self):

        ticker = Ticker(self.symbol)
        raw_dict = ticker.earnings_trend

        output_dict = {}
        output_dict['currentQtr'] = raw_dict[self.symbol]['trend'][0]['epsTrend']
        output_dict['nextQtr'] = raw_dict[self.symbol]['trend'][1]['epsTrend']
        output_dict['currentYr'] = raw_dict[self.symbol]['trend'][2]['epsTrend']
        output_dict['nextYr'] = raw_dict[self.symbol]['trend'][3]['epsTrend']

        return output_dict


### WORKS USING ALPHA VANTAGE

In [8]:
global AlphaVantageKey
#basic key AlphaVantageKey = 'XEQ9AFU8KM035KMG'
#premium api key
AlphaVantageKey = 'ZTB6U564ILR50HU3'
# @cache_request()
def fairValue_hist(ticker):
    global AlphaVantageKey

    
    yq_ticker = Ticker(ticker)
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol='+ticker+'&apikey='+AlphaVantageKey
    r = requests.get(url)
    data = r.json()
    # print(type(data['annualEarnings']))
    # display(data['fannualEarnings'])
    df = pd.DataFrame(data['annualEarnings'])
    df['fiscalDateEnding'] = pd.to_datetime(df['fiscalDateEnding'])
    filtered_values = np.where((df['fiscalDateEnding'] > '2017-01-01') & (df['fiscalDateEnding'] < '2020-01-01'))
    # display(filtered_values)
    eps1 = df.loc[filtered_values]
#     display(eps1)
    # print(filtered_values)

    testPrice = []
    reportedEPS_cols = []
    pe_cols = []
    testPrice_cols = []

    for index, row in eps1.iterrows():
        #retriving testPrice from Past #change to 30-60 days average
        testPriceDateStart = row['fiscalDateEnding'] + relativedelta(months=+3)
        testPriceDateEnd = row['fiscalDateEnding'] + relativedelta(months=+4)
        df = yq_ticker.history(period='7y', interval='1d')
        priceHistory = pd.DataFrame(df)
        priceHistory = priceHistory.reset_index()
        display(priceHistory)
#         priceHistory['date'] = priceHistory['date'].dt.strftime('%Y-%m-%d')
        print('dtypes')
        display(priceHistory.dtypes)
        priceHistory.drop(priceHistory.tail(1).index,inplace=True)
        priceHistory['date']= pd.to_datetime(priceHistory['date'])
        mask = (priceHistory['date'] > testPriceDateStart) & (priceHistory['date'] < testPriceDateEnd)
        avg_price = priceHistory.loc[mask]
        avgTestPrice = avg_price['close'].mean()
        testPrice.append(avgTestPrice)
        currentYear = row['fiscalDateEnding'].strftime("%Y")

        #making all the columns
        reportedEPS_cols.append('reportedEPS' + currentYear)
        pe_cols.append('p_e'+currentYear)
        print('reportedEPS: ', reportedEPS_cols)
        print('p_e', pe_cols)
        testPrice_cols.append('avgPrice' + currentYear)
        testdf = pd.DataFrame(testPrice)
        print('testdf')
        display(testdf)

    #building vertical dataframe which ends up getting transposed
    eps1.reset_index(drop=True)
    eps1['testPrice'] = testdf.values
    eps1['reportedEPS'] = eps1['reportedEPS'].astype(float)
    eps1['p_e'] = eps1['testPrice']/eps1['reportedEPS']
    eps1['avgp_e'] = eps1['p_e'].mean()
    #current yr estimate
    currentYrEstimate = StockDetail(ticker).get_earnings_trend()['currentYr']['current']
    eps1['currentYrEstimate'] = currentYrEstimate
    eps1['FairValue'] = eps1['avgp_e'] * eps1['currentYrEstimate']
    
    #bulding long data lists which is what we actually use for fv_df
    data = eps1['reportedEPS'].to_list()
    data.extend(eps1['testPrice'].to_list())
    data.extend(eps1['p_e'].to_list())
    data.append(eps1['currentYrEstimate'].values[0])
    data.append(eps1['FairValue'].values[0])

    #creating final long data row from lists
    fv_df_cols = reportedEPS_cols + testPrice_cols + pe_cols + ['currentYrEstimate', 'FairValue']
#     print(fv_df_cols)
    fv_df = pd.DataFrame(data = [data], columns = fv_df_cols)

    print('fv_df_currentYR: ', fv_df)
    return fv_df


fairValue_hist('aapl')

,symbol,date,open,high,low,close,volume,adjclose,dividends,splits
0,aapl,2016-06-28,23.225000,23.415001,23.035000,23.397499,161779600,21.604416,0.0,0.0
1,aapl,2016-06-29,23.492500,23.637501,23.407499,23.600000,146124000,21.791399,0.0,0.0
2,aapl,2016-06-30,23.610001,23.942499,23.575001,23.900000,143345600,22.068413,0.0,0.0
3,aapl,2016-07-01,23.872499,24.117500,23.832500,23.972500,104106000,22.135359,0.0,0.0
4,aapl,2016-07-05,23.847500,23.850000,23.615000,23.747499,110820800,21.927595,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1756,aapl,2023-06-21,184.899994,185.410004,182.589996,183.960007,49515700,183.960007,0.0,0.0
1757,aapl,2023-06-22,183.740005,187.050003,183.669998,187.000000,51245300,187.000000,0.0,0.0
1758,aapl,2023-06-23,185.550003,187.559998,185.009995,186.679993,53079300,186.679993,0.0,0.0
1759,aapl,2023-06-26,186.830002,188.050003,185.229996,185.270004,48088700,185.270004,0.0,0.0


dtypes


symbol        object
date          object
open         float64
high         float64
low          float64
close        float64
volume         int64
adjclose     float64
dividends    float64
splits       float64
dtype: object

reportedEPS:  ['reportedEPS2019']
p_e ['p_e2019']
testdf


,0
0,77.631375


...

symbol        object
date          object
open         float64
high         float64
low          float64
close        float64
volume         int64
adjclose     float64
dividends    float64
splits       float64
dtype: object

reportedEPS:  ['reportedEPS2019', 'reportedEPS2018', 'reportedEPS2017']
p_e ['p_e2019', 'p_e2018', 'p_e2017']
testdf


,0
0,77.631375
1,38.294250
2,43.680395


fv_df_currentYR:     reportedEPS2019  reportedEPS2018  reportedEPS2017  avgPrice2019  \
0             2.98             2.97              2.3     77.631375   

   avgPrice2018  avgPrice2017    p_e2019    p_e2018    p_e2017  \
0      38.29425     43.680395  26.050797  12.893687  18.991476   

   currentYrEstimate  FairValue  
0               5.98  115.48568  


,reportedEPS2019,reportedEPS2018,reportedEPS2017,avgPrice2019,avgPrice2018,avgPrice2017,p_e2019,p_e2018,p_e2017,currentYrEstimate,FairValue
0,2.98,2.97,2.3,77.631375,38.29425,43.680395,26.050797,12.893687,18.991476,5.98,115.48568


# polygon version of fairValue

In [4]:
key= 'crAsQ9cC69M4GdlCPLqLDrOaloQ8v8sk'
ticker = 'AAPL'
#works
# url = 'https://api.polygon.io/vX/reference/financials?ticker=AAPL&apiKey='+key

# url = 'https://api.polygon.io/vX/reference/financials?ticker=AAPL&filing_date.gte=2015-01-01&filing_date.lte=2024-01-01&apiKey='+key

url = 'https://api.polygon.io/vX/reference/financials?adjusted=true&ticker=AAPL&filing_date.gte=2015-01-01&filing_date.lte=2024-01-01&limit=100&order=asc&apiKey='+key
#working below this 1
# url = 'https://api.polygon.io/vX/reference/financials?ticker=AAPL&filing_date.gte=2015-01-01&filing_date.lte=2024-01-01&limit=100&order=asc&apiKey='+key
# url = 'https://api.polygon.io/vX/reference/financials?ticker=AAPL&apiKey='+key
# url = 'https://api.polygon.io/v2/reference/financials?ticker='+ticker+'limit=120&apiKey=+'key

# url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol='+ticker+'&apikey='+AlphaVantageKey
r = requests.get(url)
display(r)
data = r.json()
# print(data)
# display(data)
# print(json.dumps(data, sort_keys=True, indent=4))

df = pd.DataFrame(data)
# display(df)
df2 = pd.DataFrame(df['results'].apply(pd.Series))
display(df2)
print(type(df2['timeframe']))
# display(df2)
df2 = df2[df2['timeframe'] == 'annual']
dfTimeData = df2[['timeframe', 'fiscal_period', 'fiscal_year', 'tickers', 'start_date', 'end_date']]
display(dfTimeData)


df3 = pd.DataFrame(df2['financials'].apply(pd.Series))
df4 = pd.DataFrame(df3['income_statement'].apply(pd.Series))
dfDiluted = pd.DataFrame(df4['diluted_earnings_per_share'].apply(pd.Series))
display(dfDiluted)
# df5 = pd.DataFrame(df4['income_statement'].apply(pd.Series))
# display(df5)
df5 = pd.merge(dfDiluted, dfTimeData, left_index=True, right_index=True)

display(df5)


<Response [200]>

,id,start_date,end_date,filing_date,timeframe,fiscal_period,fiscal_year,cik,sic,tickers,company_name,source_filing_url,source_filing_file_url,financials,acceptance_datetime
0,0000320193:2015:Q1,2014-09-28,2014-12-27,2015-01-28,quarterly,Q1,2015,0000320193,3571,[AAPL],APPLE INC,https://api.polygon.io/v1/reference/sec/filing...,http://api.polygon.io/v1/reference/sec/filings...,{'comprehensive_income': {'comprehensive_incom...,NaN
1,0000320193:2015:Q2,2014-12-28,2015-03-28,2015-04-28,quarterly,Q2,2015,0000320193,3571,[AAPL],APPLE INC,https://api.polygon.io/v1/reference/sec/filing...,http://api.polygon.io/v1/reference/sec/filings...,{'comprehensive_income': {'comprehensive_incom...,NaN
2,0000320193:2015:Q3,2015-03-29,2015-06-27,2015-07-22,quarterly,Q3,2015,0000320193,3571,[AAPL],APPLE INC,https://api.polygon.io/v1/reference/sec/filing...,http://api.polygon.io/v1/reference/sec/filings...,{'comprehensive_income': {'comprehensive_incom...,NaN
3,0000320193:2015:FY,2014-09-28,2015-09-26,2015-10-28,annual,FY,2015,0000320193,3571,[AAPL],APPLE INC,https://api.polygon.io/v1/reference/sec/filing...,http://api.polygon.io/v1/reference/sec/filings...,{'balance_sheet': {'noncurrent_liabilities': {...,NaN
4,0000320193:2016:Q1,2015-09-27,2015-12-26,2016-01-27,quarterly,Q1,2016,0000320193,3571,[AAPL],APPLE INC,https://api.polygon.io/v1/reference/sec/filing...,http://api.polygon.io/v1/reference/sec/filings...,{'cash_flow_statement': {'net_cash_flow_from_f...,NaN
5,0000320193:2016:Q2,2015-12-27,2016-03-26,2016-04-27,quarterly,Q2,2016,0000320193,3571,[AAPL],APPLE INC,https://api.polygon.io/v1/reference/sec/filing...,http://api.polygon.io/v1/reference/sec/filings...,{'income_statement': {'income_tax_expense_bene...,NaN
6,0000320193:2016:Q3,2016-03-27,2016-06-25,2016-07-27,quarterly,Q3,2016,0000320193,3571,[AAPL],APPLE INC,https://api.polygon.io/v1/reference/sec/filing...,http://api.polygon.io/v1/reference/sec/filings...,{'cash_flow_statement': {'net_cash_flow': {'va...,NaN
7,0000320193:2016:FY,2015-09-27,2016-09-24,2016-10-26,annual,FY,2016,0000320193,3571,[AAPL],APPLE INC,https://api.polygon.io/v1/reference/sec/filing...,http://api.polygon.io/v1/reference/sec/filings...,{'income_statement': {'income_tax_expense_bene...,NaN
8,0000320193:2017:Q1,2016-09-25,2016-12-31,2017-02-01,quarterly,Q1,2017,0000320193,3571,[AAPL],APPLE INC,https://api.polygon.io/v1/reference/sec/filing...,http://api.polygon.io/v1/reference/sec/filings...,{'income_statement': {'preferred_stock_dividen...,NaN
9,0000320193:2017:Q2,2017-01-01,2017-04-01,2017-05-03,quarterly,Q2,2017,0000320193,3571,[AAPL],APPLE INC,https://api.polygon.io/v1/reference/sec/filing...,http://api.polygon.io/v1/reference/sec/filings...,{'income_statement': {'operating_income_loss':...,NaN


<class 'pandas.core.series.Series'>


,timeframe,fiscal_period,fiscal_year,tickers,start_date,end_date
3,annual,FY,2015,[AAPL],2014-09-28,2015-09-26
7,annual,FY,2016,[AAPL],2015-09-27,2016-09-24
11,annual,FY,2017,[AAPL],2016-09-25,2017-09-30
15,annual,FY,2018,[AAPL],2017-10-01,2018-09-29
19,annual,FY,2019,[AAPL],2018-09-30,2019-09-28
23,annual,FY,2020,[AAPL],2019-09-29,2020-09-26
27,annual,FY,2021,[AAPL],2020-09-27,2021-09-25
31,annual,FY,2022,[AAPL],2021-09-26,2022-09-24


,value,unit,label,order
3,9.22,USD / shares,Diluted Earnings Per Share,4300
7,8.31,USD / shares,Diluted Earnings Per Share,4300
11,9.21,USD / shares,Diluted Earnings Per Share,4300
15,11.91,USD / shares,Diluted Earnings Per Share,4300
19,11.89,USD / shares,Diluted Earnings Per Share,4300
23,3.28,USD / shares,Diluted Earnings Per Share,4300
27,5.61,USD / shares,Diluted Earnings Per Share,4300
31,6.11,USD / shares,Diluted Earnings Per Share,4300


,value,unit,label,order,timeframe,fiscal_period,fiscal_year,tickers,start_date,end_date
3,9.22,USD / shares,Diluted Earnings Per Share,4300,annual,FY,2015,[AAPL],2014-09-28,2015-09-26
7,8.31,USD / shares,Diluted Earnings Per Share,4300,annual,FY,2016,[AAPL],2015-09-27,2016-09-24
11,9.21,USD / shares,Diluted Earnings Per Share,4300,annual,FY,2017,[AAPL],2016-09-25,2017-09-30
15,11.91,USD / shares,Diluted Earnings Per Share,4300,annual,FY,2018,[AAPL],2017-10-01,2018-09-29
19,11.89,USD / shares,Diluted Earnings Per Share,4300,annual,FY,2019,[AAPL],2018-09-30,2019-09-28
23,3.28,USD / shares,Diluted Earnings Per Share,4300,annual,FY,2020,[AAPL],2019-09-29,2020-09-26
27,5.61,USD / shares,Diluted Earnings Per Share,4300,annual,FY,2021,[AAPL],2020-09-27,2021-09-25
31,6.11,USD / shares,Diluted Earnings Per Share,4300,annual,FY,2022,[AAPL],2021-09-26,2022-09-24


In [18]:
print(df5.dtypes)
EPS = df5[df5['fiscal_year'] == '2017']
display(EPS)

value            float64
unit              object
label             object
order              int64
timeframe         object
fiscal_period     object
fiscal_year       object
tickers           object
start_date        object
end_date          object
dtype: object


,value,unit,label,order,timeframe,fiscal_period,fiscal_year,tickers,start_date,end_date
11,9.21,USD / shares,Diluted Earnings Per Share,4300,annual,FY,2017,[AAPL],2016-09-25,2017-09-30


In [1]:
def fairValue_hist(ticker):
    global AlphaVantageKey
    global getFairValueTime
    startTime = time.time()
    
    yq_ticker = Ticker(ticker)
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol='+ticker+'&apikey='+AlphaVantageKey
    r = requests.get(url)
    data = r.json()
    # print(type(data['annualEarnings']))
    # display(data['fannualEarnings'])
    df = pd.DataFrame(data['annualEarnings'])
    df['fiscalDateEnding'] = pd.to_datetime(df['fiscalDateEnding'])
    filtered_values = np.where((df['fiscalDateEnding'] > '2017-01-01') & (df['fiscalDateEnding'] < '2020-01-01'))
    # display(filtered_values)
    eps1 = df.loc[filtered_values]
#     display(eps1)
    # print(filtered_values)

    testPrice = []
    reportedEPS_cols = []
    pe_cols = []
    testPrice_cols = []

    for index, row in eps1.iterrows():
        #retrieving testPrice from Past #change to 30-60 days average
        testPriceDateStart = row['fiscalDateEnding'] + relativedelta(months=+3)
        testPriceDateEnd = row['fiscalDateEnding'] + relativedelta(months=+4)
        df = yq_ticker.history(period='6y', interval='1d')
        priceHistory = pd.DataFrame(df)
        priceHistory = priceHistory.reset_index()
        priceHistory['date']= pd.to_datetime(priceHistory['date'])
        mask = (priceHistory['date'] > testPriceDateStart) & (priceHistory['date'] < testPriceDateEnd)
        avg_price = priceHistory.loc[mask]
        avgTestPrice = avg_price['close'].mean()
        testPrice.append(avgTestPrice)
        currentYear = row['fiscalDateEnding'].strftime("%Y")

        #making all the columns
        reportedEPS_cols.append('reportedEPS' + currentYear)
        pe_cols.append('p_e'+currentYear)
        testPrice_cols.append('avgPrice' + currentYear)
        testdf = pd.DataFrame(testPrice)
#         print('testdf')
#         display(testdf)

    #building vertical dataframe which ends up getting transposed
    eps1.reset_index(drop=True)
    eps1['testPrice'] = testdf.values
    eps1['reportedEPS'] = eps1['reportedEPS'].astype(float)
    eps1['p_e'] = eps1['testPrice']/eps1['reportedEPS']
    eps1['avgp_e'] = eps1['p_e'].mean()
    #current yr estimate
    currentYrEstimate = StockDetail(ticker).get_earnings_trend()['currentYr']['current']
    eps1['currentYrEstimate'] = currentYrEstimate
    eps1['FairValue'] = eps1['avgp_e'] * eps1['currentYrEstimate']
    
    #bulding long data lists which is what we actually use for fv_df
    data = eps1['reportedEPS'].to_list()
    data.extend(eps1['testPrice'].to_list())
    data.extend(eps1['p_e'].to_list())
    data.append(eps1['currentYrEstimate'].values[0])
    data.append(eps1['FairValue'].values[0])

    #creating final long data row from lists
    fv_df_cols = reportedEPS_cols + testPrice_cols + pe_cols + ['currentYrEstimate', 'FairValue']
#     print(fv_df_cols)
    fv_df = pd.DataFrame(data = [data], columns = fv_df_cols)

    # executionTime = (time.time() - startTime)
    # getFairValueTime = getFairValueTime + executionTime
    print('fv_df_currentYR: ', fv_df)
    return fv_df